In [1]:
import sys

import pandas as pd
import sqlite3
# from datadings.tools import yield_threaded
from tqdm import tqdm

from Harmony.models import Harmony

# from Harmony.models.clip import CLIP_VITL16
# from Harmony.models.dinov2 import vit_large
import argparse

from Harmony import utils
import Harmony.models.vision_transformer as vits

import torch
import torchvision

c:\Users\Moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [33]:
import os
import zipfile
zip_directory = "D:/data/YFCC15M/"
zips = [zip for zip in os.listdir("D:\data\YFCC15M") if zip[-4:] == ".zip"]
# zips = zips

In [43]:
image_paths = []

def list_images_in_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # List all contents and filter for image files
        image_files = [item for item in zip_ref.namelist() if item.lower().endswith(('.jpg'))]
    return image_files

for zip_file in tqdm(zips):
    zip_path = os.path.join(zip_directory, zip_file)
    image_paths.extend(list_images_in_zip(zip_path))

100%|██████████| 4098/4098 [02:20<00:00, 29.22it/s]


In [45]:
len(image_paths)

14825236

In [41]:
len(image_paths)

14825236

In [47]:
def estimate_unzipped_size(zip_path):
    total_size = 0
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for info in zip_ref.infolist():
            total_size += info.file_size  # Use `file_size` for uncompressed size
    return total_size

all_size = 0

for zip_file in zips:
    zip_path = os.path.join(zip_directory, zip_file)
    estimated_size = estimate_unzipped_size(zip_path)
    all_size += estimated_size
    # print(f"Estimated size for {zip_file} when unzipped: {estimated_size} bytes")

In [48]:
all_size

1913923736556

In [49]:
total_aggregated_size_gb = all_size / (1024**3)

In [50]:
total_aggregated_size_gb

1782.4803819470108

In [ ]:
from itertools import chain, combinations, permutations

def power_set_permutations(iterable):
    s = list(iterable)
    return list("_".join(t) for t in chain.from_iterable(permutations(t) for t in chain.from_iterable(combinations(s, r) for r in range(len(s)+1))))

# Test the function
print(power_set_permutations(['ibot', 'dino', 'mae']))

In [ ]:
it_ = torch.load("ibot_t.pth")
ht  = torch.load("harmony_t.pth")

In [ ]:
is_ = torch.load("ibot_s.pth")
hs  = torch.load("harmony_s.pth")

In [ ]:
is_same = True

# params1 = is_
params2 = hs

params1 = it_
# params2 = ht

for key in params1:
    if key in params2:
        if not torch.equal(params1[key], params2[key]):
            print(f"Difference found in parameter: {key}")
            is_same = False
    else:
        print(f"Parameter {key} not found in the second model.")
        is_same = False

for key in params2:
    if key not in params1:
        print(f"Parameter {key} not found in the first model.")
        is_same = False

if is_same:
    print("The parameters of the two models are exactly the same.")


In [ ]:
it_.values(

)

In [ ]:
hs.values()

In [ ]:
it_

In [ ]:
ht

In [ ]:
ht.values() == it_.values()

In [ ]:
args = argparse.Namespace(arch='vit_small', batch_size_per_gpu=1, clip_grad=3.0, data='CC3M:/mnt/d/data/CC3M/cc3m/', dist_url='env://', drop_path_rate=0.1, epochs=100, freeze_last_layer=1, global_crops_scale=(0.4, 1.0), gpu=0, local_crops_number=8, local_crops_scale=(0.05, 0.4), local_rank=0, lr=0.0005, min_lr=1e-06, momentum_teacher=0.996, norm_last_layer=True, num_workers=10, objective='dino', optimizer='adamw', out_dim=65536, output_dir='/mnt/c/Users/Moham/Desktop/KAUST/results', patch_size=16, rank=0, saveckp_freq=20, seed=0, teacher_temp=0.04, use_bn_in_head=False, use_fp16=True, warmup_epochs=10, warmup_teacher_temp=0.04, warmup_teacher_temp_epochs=0, weight_decay=0.04, weight_decay_end=0.4, world_size=1)

In [ ]:
checkpoint = torch.load(r"/mnt/c/Users/Moham/Desktop/KAUST/results/checkpoint0000.pth", map_location="cpu")

In [ ]:
# utils.init_distributed_mode(args)
# utils.fix_random_seeds(args.seed)

# model = Harmony(args=args)

model = vits.__dict__[args.arch](patch_size=args.patch_size, num_classes=0)

In [ ]:
print(len(checkpoint['main_vit'].keys()))

In [ ]:
checkpoint_key = "main_vit"
state_dict = torch.load(r"/mnt/c/Users/Moham/Desktop/KAUST/results/checkpoint0000.pth", map_location="cpu")

if checkpoint_key is not None and checkpoint_key in state_dict:
    print(f"Take key {checkpoint_key} in provided checkpoint dict")
    state_dict = state_dict[checkpoint_key]
# remove `module.` prefix
# state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
# # remove `backbone.` prefix induced by multicrop wrapper
# state_dict = {k.replace("backbone.", ""): v for k, v in state_dict.items()}
msg = model.load_state_dict(state_dict, strict=False)

In [ ]:
msg

In [ ]:
msg

In [ ]:
print([i for i in model.parameters()])

In [ ]:
checkpoint['model']

In [ ]:
checkpoint['teacher']

In [ ]:
pretrained_weights = "/mnt/c/Users/Moham/Desktop/KAUST/results/checkpoint.pth"
checkpoint_key = "model"
state_dict = torch.load(pretrained_weights, map_location="cpu")
if checkpoint_key is not None and checkpoint_key in state_dict:
    print(f"Take key {checkpoint_key} in provided checkpoint dict")
    state_dict = state_dict[checkpoint_key]
msg = model.load_state_dict(state_dict, strict=False)
print('Pretrained weights found at {} and loaded with msg: {}'.format(pretrained_weights, msg))

In [ ]:
model.discriminative_path.teacher

In [ ]:
model.discriminative_path.teacher.backbone

In [ ]:
for p in model.discriminative_path.teacher.backbone.parameters():
    print(p)

In [ ]:
from torchvision import transforms as pth_transforms

# train_transform = pth_transforms.Compose([
#     pth_transforms.RandomResizedCrop(224),
#     pth_transforms.RandomHorizontalFlip(),
#     pth_transforms.ToTensor(),
#     pth_transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
# ])


train_data = torchvision.datasets.ImageFolder("/mnt/e/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train")

data = torch.utils.data.DataLoader(train_data, batch_size=1)

In [ ]:
print(data.dataset[445245])

In [ ]:
data.dataset[1277363]

In [ ]:
l = 0
for i in range(data.__len__()):
    r = data.dataset[i]
    print("class:", r[1])
    print("image:", r[0])
    print(i)

In [ ]:
import os
file_path = os.path.realpath(__file__)

In [ ]:
train_data[0][1]

In [ ]:
from torchvision import datasets
from PIL import Image
import os

class ImageNet(torch.utils.data.Dataset):
    def __init__(self, root, split="train", transform=None, **kwargs):
        self.root = root
        self.transform = transform
        self.split = split

        if split == "train":
            self.data = datasets.ImageFolder(root, transform=transform)
        elif split == "val":
            self.images =  os.listdir(self.root)
            self.image_paths = [os.path.join(self.root, image) for image in self.images]
            self.labels = pd.read_csv("./Harmony/data/meta/imagenet_val_labels.csv")

    def __len__(self):
        if self.split == "train":
            return self.data.__len__()
        else:
            return len(self.images)

    def get_image_target(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        image_name = self.images[idx].split('.')[0]
        label = self.labels[self.labels['ImageId'] == image_name]['PredictionString'].item()
        return image, label

    def __getitem__(self, idx):

        if self.split == "train":
            return self.data[idx][0], self.data[idx][1]
        
        image, target = self.get_image_target(idx)
        
        if self.transform:
            image = self.transform(image)

        return image, target

In [ ]:
data = ImageNet(root="/mnt/e/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/val/", split="val")

In [ ]:
for i, t in data:
    print(t)

In [ ]:
import torch
import torch.nn as nn

from timm.models.vision_transformer import PatchEmbed, Block
from vision_transformer import Block

from utils import get_2d_sincos_pos_embed

class GenerativePath(nn.Module):
    def __init__(self, image_encoder, patch_size=16, in_chans=3,
                embed_dim=1024, decoder_embed_dim=512, decoder_depth=8, decoder_num_heads=16,
                mlp_ratio=4, norm_layer=nn.LayerNorm, norm_pix_loss=False):
        super().__init__()

        self.norm_pix_loss = norm_pix_loss
        self.image_encoder = image_encoder
        self.patch_embed = image_encoder.patch_embed
        num_patches = self.image_encoder.patch_embed.num_patches

        self.decoder_embed = nn.Linear(embed_dim, decoder_embed_dim, bias=True)
        self.mask_token = nn.Parameter(torch.zeros(1, 1, decoder_embed_dim))
        self.decoder_pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, decoder_embed_dim), requires_grad=False)  # fixed sin-cos embedding

        self.decoder_blocks = nn.ModuleList([
            Block(decoder_embed_dim, decoder_num_heads, mlp_ratio, qkv_bias=True, qk_scale=None, norm_layer=norm_layer)
            for i in range(decoder_depth)])

        self.decoder_norm = norm_layer(decoder_embed_dim)
        self.decoder_pred = nn.Linear(decoder_embed_dim, patch_size**2 * in_chans, bias=True) # decoder to patch

    def initialize_deocder_weights(self):
        
        decoder_pos_embed = get_2d_sincos_pos_embed(self.decoder_pos_embed.shape[-1], int(self.patch_embed.num_patches**.5), cls_token=True)
        self.decoder_pos_embed.data.copy_(torch.from_numpy(decoder_pos_embed).float().unsqueeze(0))

        torch.nn.init.normal_(self.mask_token, std=.02)

    def patchify(self, imgs):
        """
        imgs: (N, 3, H, W)
        x: (N, L, patch_size**2 *3)
        """
        print(self.patch_embed.patch_size)
        p = self.patch_embed.patch_size
        assert imgs.shape[2] == imgs.shape[3] and imgs.shape[2] % p == 0

        h = w = imgs.shape[2] // p
        x = imgs.reshape(shape=(imgs.shape[0], 3, h, p, w, p))
        x = torch.einsum('nchpwq->nhwpqc', x)
        x = x.reshape(shape=(imgs.shape[0], h * w, p**2 * 3))
        return x

    def unpatchify(self, x):
        """
        x: (N, L, patch_size**2 *3)
        imgs: (N, 3, H, W)
        """
        p = self.patch_embed.patch_size
        h = w = int(x.shape[1]**.5)
        assert h * w == x.shape[1]
        
        x = x.reshape(shape=(x.shape[0], h, w, p, p, 3))
        x = torch.einsum('nhwpqc->nchpwq', x)
        imgs = x.reshape(shape=(x.shape[0], 3, h * p, h * p))
        return imgs

    def random_masking(self, x, mask_ratio):
        """
        Perform per-sample random masking by per-sample shuffling.
        Per-sample shuffling is done by argsort random noise.
        x: [N, L, D], sequence
        """
        N, L, D = x.shape  # batch, length, dim
        len_keep = int(L * (1 - mask_ratio))
        
        noise = torch.rand(N, L, device=x.device)  # noise in [0, 1]
        
        # sort noise for each sample
        ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
        ids_restore = torch.argsort(ids_shuffle, dim=1)

        # keep the first subset
        ids_keep = ids_shuffle[:, :len_keep]
        x_masked = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, D))

        # generate the binary mask: 0 is keep, 1 is remove
        mask = torch.ones([N, L], device=x.device)
        mask[:, :len_keep] = 0
        # unshuffle to get the binary mask
        mask = torch.gather(mask, dim=1, index=ids_restore)

        return x_masked, mask, ids_restore

    def forward_encoder(self, x, mask_ratio):
        # embed patches
        x = self.image_encoder.patch_embed(x)

        # add pos embed w/o cls token
        x = x + self.image_encoder.pos_embed[:, 1:, :]

        # masking: length -> length * mask_ratio
        x, mask, ids_restore = self.random_masking(x, mask_ratio)

        # append cls token
        cls_token = self.image_encoder.cls_token + self.image_encoder.pos_embed[:, :1, :]
        cls_tokens = cls_token.expand(x.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # apply Transformer blocks
        for blk in self.image_encoder.blocks:
            x = blk(x)
        x = self.image_encoder.norm(x)

        return x, mask, ids_restore

    def forward_decoder(self, x, ids_restore):
        # embed tokens
        x = self.decoder_embed(x)

        # append mask tokens to sequence
        mask_tokens = self.mask_token.repeat(x.shape[0], ids_restore.shape[1] + 1 - x.shape[1], 1)
        x_ = torch.cat([x[:, 1:, :], mask_tokens], dim=1)  # no cls token
        x_ = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1, 1, x.shape[2]))  # unshuffle
        x = torch.cat([x[:, :1, :], x_], dim=1)  # append cls token

        # add pos embed
        x = x + self.decoder_pos_embed

        # apply Transformer blocks
        for blk in self.decoder_blocks:
            x = blk(x)
        x = self.decoder_norm(x)

        # predictor projection
        x = self.decoder_pred(x)

        # remove cls token
        x = x[:, 1:, :]

        return x

    def forward_loss(self, imgs, pred, mask):
        """
        imgs: [N, 3, H, W]
        pred: [N, L, p*p*3]
        mask: [N, L], 0 is keep, 1 is remove, 
        """
        target = self.patchify(imgs)
        if self.norm_pix_loss:
            mean = target.mean(dim=-1, keepdim=True)
            var = target.var(dim=-1, keepdim=True)
            target = (target - mean) / (var + 1.e-6)**.5

        loss = (pred - target) ** 2
        loss = loss.mean(dim=-1)  # [N, L], mean loss per patch

        loss = (loss * mask).sum() / mask.sum()  # mean loss on removed patches
        return loss

    def forward(self, imgs, mask_ratio=0.75):
        latent, mask, ids_restore = self.forward_encoder(imgs, mask_ratio)
        pred = self.forward_decoder(latent, ids_restore)  # [N, L, p*p*3]
        loss = self.forward_loss(imgs, pred, mask)
        return loss, pred, mask

In [ ]:
s = GenerativePath(image_encoder=model.image_encoder, embed_dim=384).cuda()
r = torch.rand([1, 3, 224, 224]).cuda()

In [ ]:
l, p, m = s(r)

In [ ]:
clip = CLIP_VITL16()

In [ ]:
import torch

In [ ]:
import sys
import os

In [ ]:
a = [f for f in os.scandir("/mnt/d/data/CC3M/cc3m/") if f.is_dir()]

In [ ]:
def save_images_in_folders(folders):
    images_paths = []
    for folder in folders:
        for filename in os.listdir(folder):
            if os.path.isfile(os.path.join(folder, filename)):

In [ ]:
paths = save_images_in_folders(a)

In [ ]:
import os
from PIL import Image
from Harmony.data.utils import SimpleTokenizer

def save_image_captions_from_folders(folders, root):
    images_paths  = []
    captions_path = []
    for folder in folders:
        for filename in os.listdir(folder):
            if ".jpg" in filename or ".png" in filename:
                images_paths.append(root + os.sep + folder.name + os.sep + filename)
            elif ".txt" in filename:
                captions_path.append(root + os.sep + folder.name + os.sep + filename)
    return images_paths, captions_path

class CC3M(torch.utils.data.Dataset):
    def __init__(self, root, transform=None, tokneizer=SimpleTokenizer(), **kwargs):
        self.root = root
        self.folders =  [f for f in os.scandir(root) if f.is_dir()]
        self.images, self.captions = save_image_captions_from_folders(self.folders, self.root)
        self.images.sort(), self.captions.sort() # sort to make sure there is correspondence
        self.transform = transform
        self.tokenizer = tokneizer

        # self.images = self.images[:10]
        # self.captions = self.captions[:10]

        assert len(self.captions) == len(self.images)
        print("Number of images loaded in CC3M are:", {self.__len__()})

    def __len__(self):
        return len(self.images)

    def get_image_caption_pair(self, idx):
        print(self.images[idx])
        print(self.captions[idx])
        image = Image.open(self.images[idx]).convert("RGB")
        caption_file = open(self.captions[idx])
        caption = caption_file.read()
        caption_file.close()

        return image, caption


    def __getitem__(self, idx):
        image, caption = self.get_image_caption_pair(idx)
        if self.transform:
            image = self.transform(image)
        if self.tokenizer:
            caption = self.tokenizer(caption)
    
        return image, caption

In [ ]:
a = CC3M(root="D:\data\CC3M\cc3m")

In [ ]:
i, t = a[100000]

In [ ]:
import tarfile

# Open the tar file
tar = tarfile.open(r'D:\data\CC3M\cc3m\00000.tar')

# Loop over each member
for member in tar.getmembers():
    print(member)
    # Extract each file as a file object
    f = tar.extractfile(member)
    if f is not None:
        # Read the contents
        content = f.read()

# Close the tar file
tar.close()
